In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

seed = 1337

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

### 코드 4-1

In [2]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(intermediate)
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
            
        return output

### 코드 4-2

In [3]:
batch_size = 2 # 한 번에 입력할 샘플 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)


### 코드 4-3

In [4]:
def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))

In [5]:
# 입력
x_input = torch.rand(batch_size, input_dim)
describe(x_input)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.8329, 0.4277, 0.4363],
        [0.9686, 0.6316, 0.8494]])


In [6]:
y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[-0.2456,  0.0723,  0.1589, -0.3294],
        [-0.3497,  0.0828,  0.3391, -0.4271]], grad_fn=<AddmmBackward>)


### 코드 4-4

In [7]:
y_output = mlp(x_input, apply_softmax=True)
describe(y_output)

타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[0.2087, 0.2868, 0.3127, 0.1919],
        [0.1832, 0.2824, 0.3649, 0.1696]], grad_fn=<SoftmaxBackward>)


### 코드 4-13

In [8]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        """
        매개변수:
            input_dim (int): 입력 벡터 크기
            hidden_dim (int): 첫 번째 Linear 층의 출력 크기
            output_dim (int): 두 번째 Linear 층의 출력 크기
        """
        super(MultilayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x_in, apply_softmax=False):
        """MLP의 정방향 계산
        
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, input_dim)입니다.
            apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그
                크로스-엔트로피 손실을 사용하려면 False로 지정해야 합니다.
        반환값:
            결과 텐서. tensor.shape은 (batch, output_dim)입니다.
        """
        intermediate = F.relu(self.fc1(x_in))
        output = self.fc2(F.dropout(intermediate, p=0.5))
        
        if apply_softmax:
            output = F.softmax(output, dim=1)
        return output

batch_size = 2 # 동시에 입력될 샘플의 개수
input_dim = 3
hidden_dim = 100
output_dim = 4

# 모델 생성
mlp = MultilayerPerceptron(input_dim, hidden_dim, output_dim)
print(mlp)

y_output = mlp(x_input, apply_softmax=False)
describe(y_output)

MultilayerPerceptron(
  (fc1): Linear(in_features=3, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=4, bias=True)
)
타입: torch.FloatTensor
크기: torch.Size([2, 4])
값: 
tensor([[-0.3282,  0.0840,  0.2370,  0.0860],
        [-0.1289, -0.0275,  0.2067,  0.3799]], grad_fn=<AddmmBackward>)


### 코드 4-14

In [9]:
batch_size = 2
one_hot_size = 10
sequence_width = 7

data = torch.randn(batch_size, one_hot_size, sequence_width)
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
intermediate1 = conv1(data)

print(data.size())
print(intermediate1.size())

torch.Size([2, 10, 7])
torch.Size([2, 16, 5])


### 코드 4-15

In [10]:
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

intermediate2 = conv2(intermediate1)
intermediate3 = conv3(intermediate2)

print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


In [11]:
y_output = intermediate3.squeeze()

print(y_output.size())

torch.Size([2, 64])


In [12]:
intermediate2.mean(dim=0).mean(dim=1).sum()

tensor(1.1735, grad_fn=<SumBackward0>)

### 코드 4-16

In [13]:
# 특성 벡터를 줄이는 방법 2
print(intermediate1.view(batch_size, -1).size())

# 특성 벡터를 줄이는 방법 2
print(torch.mean(intermediate1, dim=2).size())
# print(torch.max(intermediate1, dim=2).size())
# print(torch.sum(intermediate1, dim=2).size())

torch.Size([2, 80])
torch.Size([2, 16])


### 코드 4-22

전체 모델을 다시 만들지 않고 배치 정규화를 사용하는 방법만 소개합니다.

In [14]:
conv1 = nn.Conv1d(in_channels=one_hot_size, out_channels=16, kernel_size=3)
conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)

conv1_bn = nn.BatchNorm1d(num_features=16)
conv2_bn = nn.BatchNorm1d(num_features=32)
    
intermediate1 = conv1_bn(F.relu(conv1(data)))
intermediate2 = conv2_bn(F.relu(conv2(intermediate1)))
intermediate3 = conv3(intermediate2)

print(intermediate1.size())
print(intermediate2.size())
print(intermediate3.size())

torch.Size([2, 16, 5])
torch.Size([2, 32, 3])
torch.Size([2, 64, 1])


노트: 배치 정규화는 배치와 시퀀스 차원에 대해 통곗값을 계산합니다. 다른 말로하면 `BatchNorm1d`에 입력되는 텐서의 크기는 `(B, C, L)`입니다(여기에서 B는 배치, C는 채널, L은 길이). 각 `(B, L)` 슬라이스마다 원점에 평균을 맞춥니다. 이는 공변량 변화(covariate shift)를 줄입니다.

In [15]:
intermediate2.mean(dim=(0, 2))

tensor([-4.4703e-08,  0.0000e+00, -3.9736e-08,  5.9605e-08,  5.9605e-08,
        -3.9736e-08,  0.0000e+00,  0.0000e+00,  3.9736e-08,  3.9736e-08,
         0.0000e+00, -1.9868e-08, -3.9736e-08,  0.0000e+00,  1.9868e-08,
         1.9868e-08,  1.9868e-08,  2.9802e-08,  0.0000e+00, -3.9736e-08,
         7.9473e-08, -1.9868e-08,  3.9736e-08,  1.9868e-08,  9.9341e-09,
        -5.9605e-08, -1.9868e-08, -3.9736e-08,  3.9736e-08,  0.0000e+00,
        -1.9868e-08, -4.4703e-08], grad_fn=<MeanBackward1>)

## 보너스 코드

4장에서 합성곱을 다룹니다. 아래 코드는 여러 하이퍼파라미터 설정으로 합성곱을 만듭니다.

In [16]:
x = torch.randn(1, 2, 3, 3)
describe(x)

conv1 = nn.Conv2d(in_channels=2, out_channels=1, kernel_size=2)
describe(conv1.weight)
describe(conv1(x))

타입: torch.FloatTensor
크기: torch.Size([1, 2, 3, 3])
값: 
tensor([[[[-1.0516,  0.8223, -0.4441],
          [ 0.5651,  0.1499,  0.8558],
          [ 1.0299, -1.5958, -1.3647]],

         [[-1.7112,  0.4895,  1.9279],
          [ 2.0921,  0.8010,  0.0450],
          [ 1.0441,  0.1169, -1.2539]]]])
타입: torch.FloatTensor
크기: torch.Size([1, 2, 2, 2])
값: 
Parameter containing:
tensor([[[[-0.1961, -0.1264],
          [-0.1118, -0.2688]],

         [[-0.2124,  0.2460],
          [ 0.0133, -0.2928]]]], requires_grad=True)
타입: torch.FloatTensor
크기: torch.Size([1, 1, 2, 2])
값: 
tensor([[[[ 0.3384,  0.0781],
          [-0.0215,  0.6795]]]], grad_fn=<ThnnConv2DBackward>)


In [17]:
x = torch.randn(1, 1, 3, 3)
describe(x)

conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=2)
describe(conv1.weight)
describe(conv1(x))

타입: torch.FloatTensor
크기: torch.Size([1, 1, 3, 3])
값: 
tensor([[[[ 0.3394, -1.6743,  0.8440],
          [-1.3214, -0.4518,  1.3287],
          [ 1.1344, -2.2107, -0.3434]]]])
타입: torch.FloatTensor
크기: torch.Size([2, 1, 2, 2])
값: 
Parameter containing:
tensor([[[[ 0.0684,  0.4659],
          [-0.1554, -0.1037]]],


        [[[ 0.1786,  0.2361],
          [-0.1286, -0.1648]]]], requires_grad=True)
타입: torch.FloatTensor
크기: torch.Size([1, 2, 2, 2])
값: 
tensor([[[[-0.6095,  0.1063],
          [-0.3528,  0.8623]],

         [[ 0.3861,  0.2156],
          [ 0.3520,  1.0503]]]], grad_fn=<ThnnConv2DBackward>)
